In [2]:
import urllib
import pandas as pd
import collections
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from time import time

judges = ['AScalia', 'AMKennedy', 'CThomas','RBGinsburg', 'SGBreyer', 'JGRoberts', 'SAAlito', 'SSotomayor','EKagan']
train_cols = ['issue','issueArea','lcDispositionDirection','petitioner','respondent']

def read():
    facts = pd.read_table('cases_55-15.csv',sep='|')
    df_justice_all = pd.read_csv('SCDB_2015_02_justiceCentered_LegalProvision.csv', encoding="ISO-8859-1")
    return df_justice_all, facts

def tfvecto(list_of_facts):
    tfvec = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'))
    X = tfvec.fit_transform(list_of_facts)
    return X

def run():
    df_justice_all, facts = read()
    df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')
    cur_df = df[df['justiceName'].isin(judges)] #DF with only the current judges + Scalia
    cur_df = cur_df[cur_df.direction.notnull()] #DF where each Judge has a direction.
                                                # Can't train otherwise
    cur_df.reset_index(inplace=True)

    X = tfvecto(cur_df.facts_of_the_case)

    cross_v(X, cur_df)
    '''
    X_array = X.toarray()
    facts_df = pd.DataFrame(X_array)
    df = pd.concat([cur_df[train_cols], facts_df], axis=1)

    cross_v(df, cur_df)
    '''

def cross_v(X, cur_df):

    cur_df['predicted'] = 0
    kf = cross_validation.KFold(len(cur_df), n_folds=5, shuffle = True)
    scores = 0
    for train, test in kf:
        score, pred = model(X, cur_df, train, test)
        # if scores:
        #     for i in range(len(score)):
        #         scores[i] += score[i]
        # else:
        #     scores = score
        scores += score
        #cur_df.predicted[test] = pred
    # for score in socores:
    #     s = score/5
    #     print ('Score of CLF = ', s)
    scores = scores/5
    print ('Score of CLF = ', scores)




def model (X, cur_df, train, test):

    model_RF = RandomForestClassifier(n_jobs=-2, n_estimators=80, max_features=None)
    #model_KN = KNeighborsClassifier(n_jobs=-2, n_neighbors=10)
    #model_SVC = LinearSVC()
    #model_Logistic = LogisticRegression()
    modelo = model_RF
    t0 = time()
    print ('Fitting model...')
    modelo.fit(X.ix[train, :], cur_df.direction[train].astype(int))
    print ('Predicting')
    y = modelo.predict(X.ix[test,:])
    score = modelo.score(X.ix[test, :], cur_df.direction[test])
    print ('Score: ', score)
    print("Time: %0.3fs" % (time() - t0))
    print ('---------------')

    return score, y

In [21]:
df_justice_all, facts = read()
df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')
cur_df = df[df['justiceName'].isin(judges)] #DF with only the current judges + Scalia
cur_df = cur_df[cur_df.direction.notnull()] #DF where each Judge has a direction.
                                            # Can't train otherwise
cur_df.reset_index(inplace=True)

In [ ]:
X = tfvecto(cur_df.facts_of_the_case)

In [3]:
X_array = X.toarray()
facts_df = pd.DataFrame(X_array)

In [32]:
df = pd.concat([cur_df[train_cols+['direction']], facts_df], axis=1)

In [ ]:
df = df[df.notnull().all(axis=1)].reset_index()

In [ ]:
x = df.ix[:,df.columns != 'direction']

In [ ]:
cross_v(x, df)

In [33]:
df.shape

(11951, 14676)

In [7]:
cur_df.shape

(11951, 66)

In [13]:
df.ix[:,train_cols].head(20)

,issue,issueArea,lcDispositionDirection,petitioner,respondent
0,20150,2,NaN,28,28
1,20150,2,NaN,28,28
2,20150,2,NaN,28,28
3,20150,2,NaN,28,28
4,20150,2,NaN,28,28
5,20150,2,NaN,28,28
6,20150,2,NaN,28,28
7,20150,2,NaN,28,28
8,20150,2,NaN,28,28
9,20150,2,NaN,28,28


In [35]:
df.shape

(11888, 14677)

In [36]:
11951-11888

63

In [37]:
[1, 2, 3, 4]-[3]

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [39]:
y = df.direction

In [40]:
x.shape

(11888, 14676)

In [41]:
y.shape

(11888,)

In [43]:
cross_v(x, df)

Fitting model...
Predicting
Score:  0.763666947014
Time: 199.106s
---------------
Fitting model...
Predicting
Score:  0.763666947014
Time: 201.692s
---------------
Fitting model...
Predicting
Score:  0.756518082422
Time: 204.727s
---------------
Fitting model...
Predicting
Score:  0.771981489272
Time: 214.175s
---------------
Fitting model...
Predicting
Score:  0.769457299117
Time: 219.344s
---------------
Score of CLF =  0.765058152968


In [28]:
df[train_cols].isnull().sum()

issue                      8
issueArea                  8
lcDispositionDirection    55
petitioner                 0
respondent                 0
dtype: int64

In [11]:
cross_v(df[df.columns, cur_df)

Fitting model...


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [9]:
cross_v(X, cur_df)

Fitting model...
Predicting
Score:  0.733584274362
Time: 61.159s
---------------
Fitting model...
Predicting
Score:  0.759832635983
Time: 63.065s
---------------
Fitting model...
Predicting
Score:  0.737656903766
Time: 62.476s
---------------
Fitting model...
Predicting
Score:  0.744769874477
Time: 68.987s
---------------
Fitting model...
Predicting
Score:  0.747280334728
Time: 62.394s
---------------
Score of CLF =  0.744624804663


In [31]:
cross_v(X, cur_df)

Fitting model...


AttributeError: ix not found

In [44]:
type(cur_df)==pd.core.frame.DataFrame

True

In [41]:
type(X)==csr_matrix

NameError: name 'csr_matrix' is not defined

In [32]:
a =[1, 2, 3, 4, 5, 6, 7, 8]

In [33]:
b=[3, 6, 8]

In [34]:
[a != 3]

[True]

In [13]:
[x not in b for x in a]

[True, True, False, True, True, False, True, False]

In [14]:
[x for x in a if x not in b]

[1, 2, 4, 5, 7]

In [22]:
from sklearn.metrics import recall_score,precision_score,accuracy_score

In [38]:
[cur_df.direction[10000:10200].value_counts().index[0]]*len(cur_df.direction[10000:10200])

[2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0

In [5]:
m = ["I think you are cute", "I will never seay you are dull","She was all alone", "You think never will all"]

In [6]:
c = tfvecto(m)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vect= CountVectorizer()

In [14]:
z = vect.fit_transform(m)

In [16]:
vect.get_feature_names()

['all',
 'alone',
 'are',
 'cute',
 'dull',
 'never',
 'seay',
 'she',
 'think',
 'was',
 'will',
 'you']

In [17]:
z.toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]], dtype=int64)

In [29]:
tfvec = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'), ngram_range=(1, 3))
d = tfvec.fit_transform(cur_df.facts_of_the_case)

In [30]:
len(tfvec.get_feature_names())

301897

In [31]:
cross_v(d, cur_df)

Fitting model...


AttributeError: ix not found

In [20]:
c.toarray()

array([[ 0.        ,  0.78528828,  0.        ,  0.        ,  0.        ,
         0.6191303 ],
       [ 0.        ,  0.        ,  0.61761437,  0.48693426,  0.61761437,
         0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.70710678,  0.        ,
         0.70710678]])

In [11]:
c

<4x6 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>